In [141]:
import pickle
import numpy as np
import string
import operator
import nltk
import gensim

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc

In [5]:
data = pickle.load(open('../saved_files/clinical_notes_excluding_newborns.p', 'rb'))
label = pickle.load(open('../saved_files/mortality_excluding_newborns.p', 'rb'))
print len(data), len(label)

51113 51113


In [6]:
def custom_split(s):
    indices = []
    unique_inds = []
    splits = []
    keep_track = 0
    for i in range(len(s)):
        if s[i] == '\n':
            keep_track = i
        elif s[i] == ':':
            indices.append(keep_track)
    if len(indices) > 0:
        unique_inds.append(indices[0])
        for i in range(1,len(indices)):
            if indices[i] != indices[i-1]:
                unique_inds.append(indices[i])

        for i in range(len(unique_inds)-1):
            splits.append(s[unique_inds[i]: unique_inds[i+1]])
        splits.append(s[unique_inds[-1]:])
    return splits


In [ ]:
# admission_data = []
# admission_info = ['history of present illness:', 'hpi:', 'chief complaint:', 'past medical history:', 
#                   'major surgical or invasive procedure:']
# i = 0
# for record in data:
#     combined_info = []
#     headings = record.split('\n\n')
#     for head in headings:
#         head = head.lower()
#         for info in admission_info:
#             if info in head:
#                 token = head.split(':')
#                 combined_info.append(token[1])
#     print i
#     i+= 1
#     admission_data.append(combined_info)
# print len(admission_data)

In [8]:
admission_data2 = []
admission_info = ['history of present illness:', 'hpi:', 'chief complaint:', 'past medical history:', 
                  'major surgical or invasive procedure:', 'allergies:', 'social history:',  
                  'family history:', 'family hx:']
i = 0
for record in data:
    combined_info = ''
    headings = custom_split(record)
    for head in headings:
        head = head.lower()
        for info in admission_info:
            if info in head:
                token = head.split(':')
                combined_info += token[1] + ' '
    admission_data2.append(combined_info)

In [67]:
count = 0
for record in data:
    if len(record) == 0:
        count+=1.0
print '% records with no clinical notes:',100*count/len(data)
print '% positive labels:', 100.0*np.sum(y)/y.shape[0]

count = 0
for i, record in enumerate(admission_data2):
    if len(record) == 0 and len(data[i]) != 0:
        count+=1
print '% records with no admission data:', 100.0*count/len(data)

% records with no clinical notes: 3.9715923542
% positive labels: 11.3317551308
% records with no admission data: 0.739537886643


In [86]:
lemmatizer = WordNetLemmatizer()

def preprocess(s):
    #removing the names/dates in []
    init = -1
    s = list(s)
    for i, ch in enumerate(s):
        if ch == '[':
            init = i
        elif ch == ']' and init > 0:
            s[init:i] = ' '
            init = -1
    s = "".join(s)
    
    if len(s)>0:
        punc_list = string.punctuation+'0123456789'+'\n'
        t = string.maketrans(punc_list, " "*len(punc_list))
        s = s.translate(t)
    tokens = ''
    for word in word_tokenize(s):
        if word not in 'abcdefghijklmnopqrstuvwxyz' and word not in stopwords.words('english'):
            word = lemmatizer.lemmatize(word)
            tokens += word + ' '
    return tokens
    
#     p = string.punctuation + ' ' + '0123456789'
#     s = s.replace('\n', ' ')
#     word_list = ''
#     for token in s.split(' '):
#         token = token.strip(p)
#         if token!= '' and token not in stopwords.words('english'):
#             word_list += token + ' '
#     return word_list

# def tokenizer_better(text):
#     # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
#     if len(text)>0:
#         punc_list = string.punctuation+'0123456789'
#         t = string.maketrans(punc_list, " "*len(punc_list))
#         text = text.translate(t)
#     tokens = ''
#     for word in word_tokenize(text):
#         if word not in 'abcdefghijklmnopqrstuvwxyz':
#             tokens += word + ' '
           
#     return tokens

In [87]:
preprocessed_data = []
for record in admission_data2:
    preprocessed_data.append(preprocess(record))

In [121]:
# vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))
vectorizer = TfidfVectorizer(max_features=3000, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(preprocessed_data)

In [123]:
x = X.toarray()
y = np.array(label)
y = y[:,1]
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    stratify=y, 
                                                    test_size=0.3)


In [130]:
clf=LogisticRegression(C = 1.0, penalty = 'l2', random_state = 0)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]
print auc(y_test, y_pred)

0.8058776764031661


In [140]:
param1 = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
param2 = np.linspace(1000, 20000, 20)
max_score = 0.0 
for c in param1:
    for max_f in param2:
        val_score, test_score = grid_search(c, int(max_f))
        if val_score > max_score:
            max_score = val_score
            print val_score, test_score, c, max_f
            


0.5447554614982176 0.5533764969368226 1e-05 1000.0
0.5546557239973895 0.5547396608832582 1e-05 2000.0
0.5592123349480295 0.5487466265038433 1e-05 3000.0
0.5648362768981691 0.5523430377612465 1e-05 4000.0
0.5661105535023878 0.5821172076168375 1e-05 7000.0
0.5668772707853729 0.5619272915719993 1e-05 11000.0
0.5690448689293734 0.5340461377100904 1e-05 15000.0
0.5724654208553955 0.5530446011201007 1e-05 17000.0
0.5783826943975046 0.577221506163031 0.0001 6000.0
0.5793192803242897 0.5839299170165195 0.0001 7000.0
0.5862342525246602 0.5682345505069413 0.0001 14000.0
0.6482062746973491 0.6407921297632622 0.001 1000.0
0.6531170919935088 0.6353424271262909 0.001 4000.0
0.6550329570870588 0.6239755193493165 0.001 11000.0
0.6553125811504729 0.6358773757367391 0.001 16000.0
0.7369576197485886 0.7411626070397351 0.01 1000.0
0.7488887382753036 0.7473098243638583 0.01 2000.0
0.7513955977510514 0.7479665665453972 0.01 8000.0
0.7546825488692432 0.7311882518026116 0.01 19000.0
0.7819527250186004 0.77461

In [138]:
def grid_search(c, max_f):
    # vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))
    vectorizer = TfidfVectorizer(max_features=max_f, stop_words=stopwords.words('english'))
    X = vectorizer.fit_transform(preprocessed_data)
    x = X.toarray()
    y = np.array(label)
    y = y[:,1]
    X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                        stratify=y, 
                                                        test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                        stratify=y_train, 
                                                        test_size=0.25)
    clf=LogisticRegression(C = c, penalty = 'l2', random_state = 0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_val)
    y_pred = y_pred[:,1]
    return auc(y_val, y_pred), auc(y_test, clf.predict_proba(X_test)[:,1])


In [94]:
s = np.sum(x, axis=0)
word_list = vectorizer.get_feature_names()
word_list_sorted = [(i,word) for i, word in sorted(zip(s,word_list), key=lambda pair: -pair[0])]

In [95]:
word_list_sorted

[(114859, u'patient'),
 (62129, u'pain'),
 (58275, u'history'),
 (56075, u'year'),
 (40367, u'left'),
 (37207, u'day'),
 (37195, u'drug'),
 (35464, u'disease'),
 (34434, u'right'),
 (32034, u'denies'),
 (32021, u'known'),
 (31051, u'chest'),
 (28154, u'pt'),
 (27328, u'old'),
 (26921, u'life'),
 (26546, u'artery'),
 (26525, u'allergy'),
 (25711, u'also'),
 (25191, u'ed'),
 (24850, u'last'),
 (24172, u'status'),
 (23281, u'time'),
 (22713, u'hypertension'),
 (22593, u'prior'),
 (22399, u'ago'),
 (21744, u'per'),
 (21356, u'admission'),
 (21067, u'post'),
 (20981, u'showed'),
 (20898, u'coronary'),
 (20885, u'transferred'),
 (20643, u'given'),
 (20478, u'report'),
 (19927, u'hospital'),
 (19800, u'home'),
 (19758, u'found'),
 (19070, u'use'),
 (18713, u'past'),
 (18306, u'week'),
 (17747, u'noted'),
 (17649, u'fever'),
 (17618, u'mg'),
 (17443, u'blood'),
 (17374, u'chronic'),
 (17252, u'admitted'),
 (17210, u'abdominal'),
 (17135, u'htn'),
 (16895, u'cancer'),
 (16847, u'started'),
 (16

In [96]:
word_list_sorted2 = [(i,word) for i, word in sorted(zip(s,word_list), key=lambda pair: pair[1])]

In [97]:
word_list_sorted2

[(1342, u'aaa'),
 (2646, u'abd'),
 (2840, u'abdomen'),
 (17210, u'abdominal'),
 (1778, u'abg'),
 (1774, u'ablation'),
 (3933, u'able'),
 (1237, u'abnormal'),
 (943, u'abnormality'),
 (2108, u'abscess'),
 (1410, u'absence'),
 (8304, u'abuse'),
 (823, u'abx'),
 (521, u'ac'),
 (956, u'access'),
 (2232, u'accident'),
 (652, u'accompanied'),
 (1283, u'according'),
 (763, u'ace'),
 (324, u'acetaminophen'),
 (439, u'ache'),
 (388, u'acid'),
 (555, u'acidosis'),
 (306, u'acquired'),
 (506, u'across'),
 (239, u'activated'),
 (2326, u'active'),
 (278, u'actively'),
 (2263, u'activity'),
 (377, u'actually'),
 (8314, u'acute'),
 (714, u'acutely'),
 (299, u'acyclovir'),
 (387, u'added'),
 (1775, u'addition'),
 (1231, u'additional'),
 (745, u'additionally'),
 (999, u'adenocarcinoma'),
 (608, u'adenoma'),
 (467, u'adhesion'),
 (350, u'adhesive'),
 (328, u'adl'),
 (1258, u'adls'),
 (457, u'administered'),
 (377, u'administration'),
 (21356, u'admission'),
 (1073, u'admit'),
 (1269, u'admits'),
 (17252

In [106]:
coefs = clf.coef_[0]
word_list_sorted = [(i,word) for i, word in sorted(zip(coefs,word_list), key=lambda pair: -pair[0])]

In [107]:
word_list_sorted

[(0.43068604560931545, u'file'),
 (0.2819857855887545, u'pupil'),
 (0.27439401086124265, u'metastatic'),
 (0.271007894048383, u'measure'),
 (0.26807685998102926, u'unresponsive'),
 (0.24189315658883145, u'ray'),
 (0.24067363576222273, u'aml'),
 (0.23440808596995952, u'large'),
 (0.22973311786867764, u'cirrhosis'),
 (0.22927279715022683, u'arrest'),
 (0.22888726951190022, u'information'),
 (0.22271196578859756, u'intubated'),
 (0.22008815916027083, u'myeloma'),
 (0.2106521545637288, u'paracentesis'),
 (0.20871272788976405, u'jaundice'),
 (0.2068776544488099, u'lived'),
 (0.2058966040446391, u'leukemia'),
 (0.20315399252555053, u'decided'),
 (0.20078420761677096, u'arterial'),
 (0.20023090546023864, u'intubation'),
 (0.17562182094384582, u'diffuse'),
 (0.1741170998770405, u'stem'),
 (0.16572857420152098, u'poor'),
 (0.1559586250806068, u'platelet'),
 (0.15581305393495104, u'shift'),
 (0.15535242556805573, u'hemorrhage'),
 (0.15212656180440234, u'lymphoma'),
 (0.1486189583723697, u'chf'),